In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [10]:
df = pd.read_csv("aug_train.csv")

In [11]:
df.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [12]:
# Encode ordinal feature using mapping to transform categorical features into numarical features

# Making Dictionaries of ordinal features

relevent_experience_map = {
    'Has relevent experience':  1,
    'No relevent experience':    0
}

experience_map = {
    '<1'      :    0,
    '1'       :    1,
    '2'       :    2,
    '3'       :    3,
    '4'       :    4,
    '5'       :    5,
    '6'       :    6,
    '7'       :    7,
    '8'       :    8,
    '9'       :    9,
    '10'      :    10,
    '11'      :    11,
    '12'      :    12,
    '13'      :    13,
    '14'      :    14,
    '15'      :    15,
    '16'      :    16,
    '17'      :    17,
    '18'      :    18,
    '19'      :    19,
    '20'      :    20,
    '>20'     :    21
}

last_new_job_map = {
    'never'        :    0,
    '1'            :    1,
    '2'            :    2,
    '3'            :    3,
    '4'            :    4,
    '>4'           :    5
}

In [13]:
# Transform categorical features into numarical features

def encode(df_pre):
    df_pre['relevent_experience'] = df_pre['relevent_experience'].map(relevent_experience_map)
    df_pre['last_new_job'] = df_pre['last_new_job'].map(last_new_job_map)
    df_pre['experience'] = df_pre['experience'].map(experience_map)
    '''
    #encoding city feature using label encoder
    lb_en = LabelEncoder()

    df_pre.loc[:,'city'] = lb_en.fit_transform(df_pre.loc[:,'city'])
    '''
    return df_pre

df = encode(df)
df.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,1,no_enrollment,Graduate,STEM,21.0,NaN,NaN,1.0,36,1.0
1,29725,city_40,0.776,Male,0,no_enrollment,Graduate,STEM,15.0,50-99,Pvt Ltd,5.0,47,0.0
2,11561,city_21,0.624,NaN,0,Full time course,Graduate,STEM,5.0,NaN,NaN,0.0,83,0.0
3,33241,city_115,0.789,NaN,0,NaN,Graduate,Business Degree,0.0,NaN,Pvt Ltd,0.0,52,1.0
4,666,city_162,0.767,Male,1,no_enrollment,Masters,STEM,21.0,50-99,Funded Startup,4.0,8,0.0


# Basic Pipeline

In [14]:
# sets of columns to be transformed in different ways

num_cols = ['city_development_index','relevent_experience', 'experience','last_new_job', 'training_hours']
cat_cols = ['gender', 'enrolled_university', 'education_level', 'major_discipline', 'company_size', 'company_type']

In [15]:
# Divide train and test sets

from sklearn.model_selection import train_test_split

X = df[num_cols+cat_cols]
y = df['target']

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [16]:
# Create pipelines for numerical and categorical features

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline

num_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('scale',MinMaxScaler())
])

cat_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('one-hot',OneHotEncoder(handle_unknown='ignore', sparse=False))
])
num_pipeline

Pipeline(steps=[('impute', SimpleImputer()), ('scale', MinMaxScaler())])

In [17]:
cat_pipeline

Pipeline(steps=[('impute', SimpleImputer(strategy='most_frequent')),
                ('one-hot',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [18]:
# Create ColumnTransformer to apply pipeline for each column type

from sklearn.compose import ColumnTransformer

col_trans = ColumnTransformer(transformers=[
    ('num_pipeline',num_pipeline,num_cols),
    ('cat_pipeline',cat_pipeline,cat_cols)
    ],
    remainder='drop',
    n_jobs=-1)
col_trans

ColumnTransformer(n_jobs=-1,
                  transformers=[('num_pipeline',
                                 Pipeline(steps=[('impute', SimpleImputer()),
                                                 ('scale', MinMaxScaler())]),
                                 ['city_development_index',
                                  'relevent_experience', 'experience',
                                  'last_new_job', 'training_hours']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('one-hot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['gender', 'enrolled_university',
                                  'education_level', 'major_discipline',
                                  'company_size', 'company_type'])])

In [19]:
# Add a model to a final pipeline

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0)

clf_pipeline = Pipeline(steps=[
    ('col_trans', col_trans),
    ('model', clf)
])
clf_pipeline

Pipeline(steps=[('col_trans',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['city_development_index',
                                                   'relevent_experience',
                                                   'experience', 'last_new_job',
                                                   'training_hours']),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['gender',
                                                   'enrolled_university',
                                                   'education_level',
                                                   'major_discipline',
                                                   'company_size',
                                                   'company_type'])])),
                ('model', LogisticRegression(random_state=0))])

In [20]:
# Display pipeline

from sklearn import set_config
set_config(display='diagram')

display(clf_pipeline)

# if desired, set display back to the default
# set_config(display='text')

Pipeline(steps=[('col_trans',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['city_development_index',
                                                   'relevent_experience',
                                                   'experience', 'last_new_job',
                                                   'training_hours']),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['gender',
                                                   'enrolled_university',
                                                   'education_level',
                                                   'major_discipline',
                                                   'company_size',
                                                   'company_type'])])),
                ('model', LogisticRegression(random_state=0))])

In [21]:
clf_pipeline.get_params()

{'memory': None,
 'steps': [('col_trans', ColumnTransformer(n_jobs=-1,
                     transformers=[('num_pipeline',
                                    Pipeline(steps=[('impute', SimpleImputer()),
                                                    ('scale', MinMaxScaler())]),
                                    ['city_development_index',
                                     'relevent_experience', 'experience',
                                     'last_new_job', 'training_hours']),
                                   ('cat_pipeline',
                                    Pipeline(steps=[('impute',
                                                     SimpleImputer(strategy='most_frequent')),
                                                    ('one-hot',
                                                     OneHotEncoder(handle_unknown='ignore',
                                                                   sparse=False))]),
                                    ['gender', 'enroll

In [22]:
# Pass data through Pipeline

clf_pipeline.fit(X_train, y_train)
# preds = clf_pipeline.predict(X_test)
score = clf_pipeline.score(X_test, y_test)
print(f"Model score: {score}")

Model score: 0.7643528183716075


In [23]:
clf_pipeline.predict(X_test)

array([0., 0., 0., ..., 0., 0., 1.])

In [24]:
clf_pipeline.predict_proba(X_test)

array([[0.8779305 , 0.1220695 ],
       [0.50282814, 0.49717186],
       [0.92111208, 0.07888792],
       ...,
       [0.80224512, 0.19775488],
       [0.6083822 , 0.3916178 ],
       [0.40010057, 0.59989943]])

In [25]:
clf_pipeline

Pipeline(steps=[('col_trans',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['city_development_index',
                                                   'relevent_experience',
                                                   'experience', 'last_new_job',
                                                   'training_hours']),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['gender',
                                                   'enrolled_university',
                                                   'education_level',
                                                   'major_discipline',
                                                   'company_size',
                                                   'company_type'])])),
                ('model', LogisticRegression(random_state=0))])

In [26]:
# Get feature name after pipeline (change column names only from one-hot encoder of categorical columns)

new_cat_cols = clf_pipeline.named_steps["col_trans"].named_transformers_["cat_pipeline"]\
                .named_steps["one-hot"].get_feature_names_out(cat_cols)
all_cols = np.concatenate([num_cols, new_cat_cols])
new_cat_cols, all_cols

(array(['gender_Female', 'gender_Male', 'gender_Other',
        'enrolled_university_Full time course',
        'enrolled_university_Part time course',
        'enrolled_university_no_enrollment', 'education_level_Graduate',
        'education_level_High School', 'education_level_Masters',
        'education_level_Phd', 'education_level_Primary School',
        'major_discipline_Arts', 'major_discipline_Business Degree',
        'major_discipline_Humanities', 'major_discipline_No Major',
        'major_discipline_Other', 'major_discipline_STEM',
        'company_size_10/49', 'company_size_100-500',
        'company_size_1000-4999', 'company_size_10000+',
        'company_size_50-99', 'company_size_500-999',
        'company_size_5000-9999', 'company_size_<10',
        'company_type_Early Stage Startup', 'company_type_Funded Startup',
        'company_type_NGO', 'company_type_Other',
        'company_type_Public Sector', 'company_type_Pvt Ltd'], dtype=object),
 array(['city_development_

In [27]:
clf_pipeline.named_steps["col_trans"].named_transformers_["cat_pipeline"]\
                .named_steps["one-hot"].get_feature_names_out(cat_cols)

array(['gender_Female', 'gender_Male', 'gender_Other',
       'enrolled_university_Full time course',
       'enrolled_university_Part time course',
       'enrolled_university_no_enrollment', 'education_level_Graduate',
       'education_level_High School', 'education_level_Masters',
       'education_level_Phd', 'education_level_Primary School',
       'major_discipline_Arts', 'major_discipline_Business Degree',
       'major_discipline_Humanities', 'major_discipline_No Major',
       'major_discipline_Other', 'major_discipline_STEM',
       'company_size_10/49', 'company_size_100-500',
       'company_size_1000-4999', 'company_size_10000+',
       'company_size_50-99', 'company_size_500-999',
       'company_size_5000-9999', 'company_size_<10',
       'company_type_Early Stage Startup', 'company_type_Funded Startup',
       'company_type_NGO', 'company_type_Other',
       'company_type_Public Sector', 'company_type_Pvt Ltd'], dtype=object)

In [28]:
clf_pipeline.named_steps["col_trans"].named_transformers_["cat_pipeline"]\
                .named_steps["one-hot"].get_feature_names_out()

array(['x0_Female', 'x0_Male', 'x0_Other', 'x1_Full time course',
       'x1_Part time course', 'x1_no_enrollment', 'x2_Graduate',
       'x2_High School', 'x2_Masters', 'x2_Phd', 'x2_Primary School',
       'x3_Arts', 'x3_Business Degree', 'x3_Humanities', 'x3_No Major',
       'x3_Other', 'x3_STEM', 'x4_10/49', 'x4_100-500', 'x4_1000-4999',
       'x4_10000+', 'x4_50-99', 'x4_500-999', 'x4_5000-9999', 'x4_<10',
       'x5_Early Stage Startup', 'x5_Funded Startup', 'x5_NGO',
       'x5_Other', 'x5_Public Sector', 'x5_Pvt Ltd'], dtype=object)

In [29]:
# Get model coef

coefs = clf_pipeline.named_steps["model"].coef_.flatten()
coef = pd.DataFrame(zip(all_cols, coefs), columns=["feature", "coef"])
coef["abs_coef"] = coef["coef"].apply(lambda x: abs(x))
coef["colors"] = coef["coef"].apply(lambda x: "green" if x > 0 else "red")
coef = coef.sort_values("abs_coef", ascending=False)

In [30]:
# Save pipeline
import joblib

joblib.dump(clf_pipeline,"pipe.joblib")

['pipe.joblib']

In [31]:
# Load pipeline when you want to use
same_pipe = joblib.load("pipe.joblib")
same_pipe

Pipeline(steps=[('col_trans',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['city_development_index',
                                                   'relevent_experience',
                                                   'experience', 'last_new_job',
                                                   'training_hours']),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['gender',
                                                   'enrolled_university',
                                                   'education_level',
                                                   'major_discipline',
                                                   'company_size',
                                                   'company_type'])])),
                ('model', LogisticRegression(random_state=0))])

# Add Pipeline to Grid Search

In [32]:
# see parameter that can be tuned

clf_pipeline.get_params()

{'memory': None,
 'steps': [('col_trans', ColumnTransformer(n_jobs=-1,
                     transformers=[('num_pipeline',
                                    Pipeline(steps=[('impute', SimpleImputer()),
                                                    ('scale', MinMaxScaler())]),
                                    ['city_development_index',
                                     'relevent_experience', 'experience',
                                     'last_new_job', 'training_hours']),
                                   ('cat_pipeline',
                                    Pipeline(steps=[('impute',
                                                     SimpleImputer(strategy='most_frequent')),
                                                    ('one-hot',
                                                     OneHotEncoder(handle_unknown='ignore',
                                                                   sparse=False))]),
                                    ['gender', 'enroll

In [33]:
clf_pipeline.set_params(model__C = 1)

Pipeline(steps=[('col_trans',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['city_development_index',
                                                   'relevent_experience',
                                                   'experience', 'last_new_job',
                                                   'training_hours']),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['gender',
                                                   'enrolled_university',
                                                   'education_level',
                                                   'major_discipline',
                                                   'company_size',
                                                   'company_type'])])),
                ('model', LogisticRegression(C=1, random_state=0))])

In [34]:
# Set tuning parameters

grid_params = {
    'model__penalty' : ['none', 'l2'],
    'model__C' : np.logspace(-4, 4, 20)
}

In [35]:
# Grid search

from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(clf_pipeline, grid_params, scoring='accuracy')
gs.fit(X_train, y_train)

print("Best Score of train set: "+str(gs.best_score_))
print("Best parameter set: "+str(gs.best_params_))
print("Test Score: "+str(gs.score(X_test,y_test)))

Best Score of train set: 0.7679762144192634
Best parameter set: {'model__C': 4.281332398719396, 'model__penalty': 'l2'}
Test Score: 0.7643528183716075


# Pipeline skip a step

In [36]:
# Add a step 'std_scale' in num_pipeline

from sklearn.preprocessing import StandardScaler

num_pipeline2 = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('minmax_scale',MinMaxScaler()),
    ('std_scale', StandardScaler()),
])

col_trans2 = ColumnTransformer(transformers=[
    ('num_pipeline',num_pipeline2,num_cols),
    ('cat_pipeline',cat_pipeline,cat_cols)
    ],
    remainder='drop',
    n_jobs=-1)

clf_pipeline2 = Pipeline(steps=[
    ('col_trans', col_trans2),
    ('model', clf)
])

In [37]:
clf_pipeline2

Pipeline(steps=[('col_trans',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('minmax_scale',
                                                                   MinMaxScaler()),
                                                                  ('std_scale',
                                                                   StandardScaler())]),
                                                  ['city_development_index',
                                                   'relevent_experience',
                                                   'experience', 'last_new_job',
                                                   'training_hours']),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['gender',
                                                   'enrolled_university',
                                                   'education_level',
                                                   'major_discipline',
                                                   'company_size',
                                                   'company_type'])])),
                ('model', LogisticRegression(C=1, random_state=0))])

In [38]:
grid_step_params = [{'col_trans__num_pipeline__minmax_scale': ['passthrough']},
                    {'col_trans__num_pipeline__std_scale': ['passthrough']}]

In [39]:
# Grid search
gs2 = GridSearchCV(clf_pipeline2, grid_step_params, scoring='accuracy')
gs2.fit(X_train, y_train)

print("Best Score of train set: "+str(gs2.best_score_))
print("Best parameter set: "+str(gs2.best_params_))
print("Test Score: "+str(gs2.score(X_test,y_test)))

Best Score of train set: 0.7680414247086127
Best parameter set: {'col_trans__num_pipeline__minmax_scale': 'passthrough'}
Test Score: 0.7643528183716075


In [ ]:
# Both parameter and step

In [40]:
# Set tuning parameters in grid params
# Set step parameters in grid_step_params

grid_params = {'model__penalty' : ['none', 'l2'],
    'model__C' : np.logspace(-4, 4, 20)}

grid_step_params2 = [{**{'col_trans__num_pipeline__minmax_scale': ['passthrough']}, **grid_params},
                    {**{'col_trans__num_pipeline__std_scale': ['passthrough']}, **grid_params}]

# Grid search
gs3 = GridSearchCV(clf_pipeline2, grid_step_params2, scoring='accuracy')
gs3.fit(X_train, y_train)

print("Best Score of train set: "+str(gs3.best_score_))
print("Best parameter set: "+str(gs3.best_params_))
print("Test Score: "+str(gs3.score(X_test,y_test)))

Best Score of train set: 0.7681719091355061
Best parameter set: {'col_trans__num_pipeline__minmax_scale': 'passthrough', 'model__C': 0.23357214690901212, 'model__penalty': 'l2'}
Test Score: 0.7638308977035491


In [41]:
# see all grid search result
pd.DataFrame(gs3.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_col_trans__num_pipeline__minmax_scale,param_model__C,param_model__penalty,param_col_trans__num_pipeline__std_scale,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.335139,0.077765,0.053570,0.010727,passthrough,0.0001,none,NaN,{'col_trans__num_pipeline__minmax_scale': 'pas...,0.764514,0.773573,0.765416,0.767700,0.768026,0.767846,0.003158,14
1,0.195286,0.020614,0.048625,0.010538,passthrough,0.0001,l2,NaN,{'col_trans__num_pipeline__minmax_scale': 'pas...,0.750489,0.750734,0.750734,0.750734,0.750408,0.750620,0.000142,77
2,0.228594,0.018352,0.034286,0.002491,passthrough,0.000264,none,NaN,{'col_trans__num_pipeline__minmax_scale': 'pas...,0.764514,0.773573,0.765416,0.767700,0.768026,0.767846,0.003158,14
3,0.145184,0.005983,0.035319,0.001291,passthrough,0.000264,l2,NaN,{'col_trans__num_pipeline__minmax_scale': 'pas...,0.750489,0.750734,0.751713,0.751060,0.750734,0.750946,0.000424,75
4,0.220933,0.009094,0.041672,0.003216,passthrough,0.000695,none,NaN,{'col_trans__num_pipeline__minmax_scale': 'pas...,0.764514,0.773573,0.765416,0.767700,0.768026,0.767846,0.003158,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.300845,0.064623,0.044289,0.005183,NaN,1438.449888,l2,passthrough,{'col_trans__num_pipeline__std_scale': 'passth...,0.764514,0.773573,0.765416,0.767700,0.768026,0.767846,0.003158,14
76,0.210121,0.007150,0.038374,0.000979,NaN,3792.690191,none,passthrough,{'col_trans__num_pipeline__std_scale': 'passth...,0.764514,0.773573,0.765416,0.767700,0.768026,0.767846,0.003158,14
77,0.209432,0.013014,0.037291,0.002606,NaN,3792.690191,l2,passthrough,{'col_trans__num_pipeline__std_scale': 'passth...,0.764514,0.773573,0.765416,0.767700,0.768026,0.767846,0.003158,14
78,0.207397,0.005774,0.037386,0.002263,NaN,10000.0,none,passthrough,{'col_trans__num_pipeline__std_scale': 'passth...,0.764514,0.773573,0.765416,0.767700,0.768026,0.767846,0.003158,14


# Add custom transformation in a pipeline

In [47]:
df = pd.read_csv("aug_train.csv")
df.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [48]:
# Create custom transformation class

from sklearn.base import TransformerMixin

class Encode(TransformerMixin):
    # Encode ordinal feature using mapping to transform categorical features into numarical features
    def __init__(self):
        print("---------init----------")
        # Making Dictionaries of ordinal features
        self.relevent_experience_map = {
            'Has relevent experience':  1,
            'No relevent experience':    0
        }

        self.experience_map = {
            '<1'      :    0,
            '1'       :    1,
            '2'       :    2,
            '3'       :    3,
            '4'       :    4,
            '5'       :    5,
            '6'       :    6,
            '7'       :    7,
            '8'       :    8,
            '9'       :    9,
            '10'      :    10,
            '11'      :    11,
            '12'      :    12,
            '13'      :    13,
            '14'      :    14,
            '15'      :    15,
            '16'      :    16,
            '17'      :    17,
            '18'      :    18,
            '19'      :    19,
            '20'      :    20,
            '>20'     :    21
        }

        self.last_new_job_map = {
            'never'        :    0,
            '1'            :    1,
            '2'            :    2,
            '3'            :    3,
            '4'            :    4,
            '>4'           :    5
        }

    def fit(self, df, y = None):
        print("---------fit----------")
        return self

    def transform(self, df, y = None):
        print("---------transform----------")
        df_pre = df.copy()
        df_pre['relevent_experience'] = df_pre['relevent_experience'].map(self.relevent_experience_map)
        df_pre['last_new_job'] = df_pre['last_new_job'].map(self.last_new_job_map)
        df_pre['experience'] = df_pre['experience'].map(self.experience_map)
        return df_pre


In [49]:
from sklearn.pipeline import Pipeline

encode_pipeline = Pipeline(steps=[('encode', Encode())])

---------init----------


In [50]:
encode_pipeline.fit(df)

---------fit----------


Pipeline(steps=[('encode', <__main__.Encode object at 0x7cd3982ff8b0>)])

In [51]:
encode_pipeline.transform(df)

---------transform----------


,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,1,no_enrollment,Graduate,STEM,21.0,NaN,NaN,1.0,36,1.0
1,29725,city_40,0.776,Male,0,no_enrollment,Graduate,STEM,15.0,50-99,Pvt Ltd,5.0,47,0.0
2,11561,city_21,0.624,NaN,0,Full time course,Graduate,STEM,5.0,NaN,NaN,0.0,83,0.0
3,33241,city_115,0.789,NaN,0,NaN,Graduate,Business Degree,0.0,NaN,Pvt Ltd,0.0,52,1.0
4,666,city_162,0.767,Male,1,no_enrollment,Masters,STEM,21.0,50-99,Funded Startup,4.0,8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19153,7386,city_173,0.878,Male,0,no_enrollment,Graduate,Humanities,14.0,NaN,NaN,1.0,42,1.0
19154,31398,city_103,0.920,Male,1,no_enrollment,Graduate,STEM,14.0,NaN,NaN,4.0,52,1.0
19155,24576,city_103,0.920,Male,1,no_enrollment,Graduate,STEM,21.0,50-99,Pvt Ltd,4.0,44,0.0
19156,5756,city_65,0.802,Male,1,no_enrollment,High School,NaN,0.0,500-999,Pvt Ltd,2.0,97,0.0


# Multiple classifier in a pipeline

In [64]:
df = pd.read_csv("aug_train.csv")
df.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


# Full pipeline

In [70]:
# Create custom transformation class

from sklearn.base import TransformerMixin

class Encode(TransformerMixin):
    # Encode ordinal feature using mapping to transform categorical features into numarical features
    def __init__(self):
        print("---------init----------")
        # Making Dictionaries of ordinal features
        self.relevent_experience_map = {
            'Has relevent experience':  1,
            'No relevent experience':    0
        }

        self.experience_map = {
            '<1'      :    0,
            '1'       :    1,
            '2'       :    2,
            '3'       :    3,
            '4'       :    4,
            '5'       :    5,
            '6'       :    6,
            '7'       :    7,
            '8'       :    8,
            '9'       :    9,
            '10'      :    10,
            '11'      :    11,
            '12'      :    12,
            '13'      :    13,
            '14'      :    14,
            '15'      :    15,
            '16'      :    16,
            '17'      :    17,
            '18'      :    18,
            '19'      :    19,
            '20'      :    20,
            '>20'     :    21
        }

        self.last_new_job_map = {
            'never'        :    0,
            '1'            :    1,
            '2'            :    2,
            '3'            :    3,
            '4'            :    4,
            '>4'           :    5
        }

    def fit(self, df, y = None):
        print("---------fit----------")
        return self

    def transform(self, df, y = None):
        print("---------transform----------")
        df_pre = df.copy()
        df_pre['relevent_experience'] = df_pre['relevent_experience'].map(self.relevent_experience_map)
        df_pre['last_new_job'] = df_pre['last_new_job'].map(self.last_new_job_map)
        df_pre['experience'] = df_pre['experience'].map(self.experience_map)
        return df_pre

In [71]:
from sklearn.base import BaseEstimator
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

class ClfSwitcher(BaseEstimator):

    def __init__(self, estimator = LogisticRegression()):
        """
        A Custom BaseEstimator that can switch between classifiers.
        :param estimator: sklearn object - The classifier
        """
        self.estimator = estimator

    def fit(self, X, y=None, **kwargs):
        self.estimator.fit(X, y)
        return self

    def predict(self, X, y=None):
        return self.estimator.predict(X)

    def predict_proba(self, X):
        return self.estimator.predict_proba(X)

    def score(self, X, y):
        return self.estimator.score(X, y)

In [72]:
# sets of columns to be transformed in different ways

num_cols = ['city_development_index','relevent_experience', 'experience','last_new_job', 'training_hours']
cat_cols = ['gender', 'enrolled_university', 'education_level', 'major_discipline', 'company_size', 'company_type']

In [73]:
# Divide train and test sets

from sklearn.model_selection import train_test_split

X = df[num_cols+cat_cols]
y = df['target']

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [74]:
# Create pipelines for numerical and categorical features

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline

num_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('scale',MinMaxScaler())
])

cat_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('one-hot',OneHotEncoder(handle_unknown='ignore', sparse=False))
])

In [75]:
# Create ColumnTransformer to apply pipeline for each column type

from sklearn.compose import ColumnTransformer

col_trans = ColumnTransformer(transformers=[
    ('num_pipeline',num_pipeline,num_cols),
    ('cat_pipeline',cat_pipeline,cat_cols)
    ],
    remainder='drop',
    n_jobs=-1)

In [76]:
# Add a model to a final pipeline

full_pipeline = Pipeline(steps=[
    ('Encode', Encode()),
    ('col_trans', col_trans),
    ('model', ClfSwitcher())
])
full_pipeline

---------init----------


Pipeline(steps=[('Encode', <__main__.Encode object at 0x7cd3908551e0>),
                ('col_trans',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['city_development_index',
                                                   'relevent_experience',
                                                   'experience', 'last_new_job',
                                                   'training_hours']),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['gender',
                                                   'enrolled_university',
                                                   'education_level',
                                                   'major_discipline',
                                                   'company_size',
                                                   'company_type'])])),
                ('model', ClfSwitcher())])

In [77]:
# Display pipeline

from sklearn import set_config
set_config(display='diagram')

display(full_pipeline)

# if desired, set display back to the default
# set_config(display='text')

Pipeline(steps=[('Encode', <__main__.Encode object at 0x7cd3908551e0>),
                ('col_trans',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['city_development_index',
                                                   'relevent_experience',
                                                   'experience', 'last_new_job',
                                                   'training_hours']),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['gender',
                                                   'enrolled_university',
                                                   'education_level',
                                                   'major_discipline',
                                                   'company_size',
                                                   'company_type'])])),
                ('model', ClfSwitcher())])

In [78]:
grid_params = [
    {'model__estimator': [LogisticRegression()]},
    {'model__estimator': [SVC(gamma='auto')]}
]

In [79]:
# Grid search

from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(full_pipeline, grid_params, scoring='accuracy')
gs.fit(X_train, y_train)

print("Best Score of train set: "+str(gs.best_score_))
print("Best parameter set: "+str(gs.best_params_))
print("Test Score: "+str(gs.score(X_test,y_test)))

---------fit----------
---------transform----------
---------transform----------
---------fit----------
---------transform----------
---------transform----------
---------fit----------
---------transform----------
---------transform----------
---------fit----------
---------transform----------
---------transform----------
---------fit----------
---------transform----------
---------transform----------
---------fit----------
---------transform----------
---------transform----------
---------fit----------
---------transform----------
---------transform----------
---------fit----------
---------transform----------
---------transform----------
---------fit----------
---------transform----------
---------transform----------
---------fit----------
---------transform----------
---------transform----------
---------fit----------
---------transform----------
Best Score of train set: 0.7671273101074885
Best parameter set: {'model__estimator': LogisticRegression()}
---------transform----------
Te

In [80]:
pd.DataFrame(gs.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__estimator,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.568791,0.597333,0.046576,0.005324,LogisticRegression(),{'model__estimator': LogisticRegression()},0.773973,0.758564,0.765742,0.768352,0.769005,0.767127,0.005042,1
1,6.763996,0.504054,1.834447,0.422584,SVC(gamma='auto'),{'model__estimator': SVC(gamma='auto')},0.751468,0.742577,0.752365,0.748124,0.758564,0.750620,0.005250,2
